## Baihan Lin, Rabadan Lab, Columbia University

In [2]:
%matplotlib inline

In [3]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torch.autograd import Variable


In [ ]:
p = 205437 
n = 100
d = 20
m = 10

Q = 0*np.ndarray((m,d,4))
K = 0*np.ndarray((p,d,4))
V = 0*np.ndarray((p,d,4))


In [4]:
# Hyperparameters
input_size = 784  # Our images are 28px by 28px in size
num_classes = 10  # We have handwritten digits from 0 - 9
num_epochs = 5  # Number of epochs
batch_size = 100  # Batch size
learning_rate = 0.001  # Learning rate

transfm = transforms.ToTensor()  # Transform the dataset objects to tensors

# MNIST dataset - images and labels
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transfm,
                               download=True)

test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transfm)

# Input pipeline
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


Processing...
Done!


In [5]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

In [6]:
model = LogisticRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
# Training the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))  # Images flattened into 1D tensors
        labels = Variable(labels)  # Labels 
        
        # Forward -> Backprop -> Optimize
        optimizer.zero_grad()  # Manually zero the gradient buffers
        outputs = model(images)  # Predict the class using the test set
        loss = criterion(outputs, labels)  # Compute the loss given the predicted label
                                           # and actual label
        
        loss.backward()  # Compute the error gradients
        optimizer.step()  # Optimize the model via Stochastic Gradient Descent
        
        if (i + 1) % 100 == 0:
            print("Epoch {}, loss :{}".format(epoch + 1, loss.data[0]))

Epoch 1, loss :2.267711639404297
Epoch 1, loss :2.148483991622925
Epoch 1, loss :2.058659076690674
Epoch 1, loss :1.9498668909072876
Epoch 1, loss :1.8805489540100098
Epoch 1, loss :1.7623120546340942
Epoch 2, loss :1.6965137720108032
Epoch 2, loss :1.6472450494766235
Epoch 2, loss :1.622676968574524
Epoch 2, loss :1.5895267724990845
Epoch 2, loss :1.5569528341293335
Epoch 2, loss :1.45692777633667
Epoch 3, loss :1.506378173828125
Epoch 3, loss :1.4878369569778442
Epoch 3, loss :1.3821254968643188
Epoch 3, loss :1.309884786605835
Epoch 3, loss :1.2609459161758423
Epoch 3, loss :1.2971638441085815
Epoch 4, loss :1.2796509265899658
Epoch 4, loss :1.1881992816925049
Epoch 4, loss :1.2206101417541504
Epoch 4, loss :1.087363839149475
Epoch 4, loss :1.2598435878753662
Epoch 4, loss :1.2453800439834595
Epoch 5, loss :1.153346061706543
Epoch 5, loss :1.0692975521087646
Epoch 5, loss :1.0982111692428589
Epoch 5, loss :1.036110758781433
Epoch 5, loss :1.034468412399292
Epoch 5, loss :0.874213159

In [8]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28 * 28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
 
print('Accuracy: {}%'.format(100 * correct / total))

Accuracy: 82.57%
